Notebook accompanying the paper

Note that this notebook requires installing: pysteps, matplotlib, cartopy

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import string
import pickle
import numpy as np
import os
import sys
from pathlib import Path
from scipy.stats import pearsonr
from pysteps.verification.spatialscores import fss
import matplotlib.pyplot as plt

HOME = Path(os.getcwd()).parents[0]

sys.path.insert(1, str(HOME))

from fssim.scoring import get_convoluted_fractions, calculate_fractions_SSIM, calculate_correlation

latitude_range = np.arange(-11.95, 15.95, 0.1)
longitude_range = np.arange(25.05, 49.05, 0.1)



## Plots of correlation and FSS

In [ ]:
with open('examples.pkl', 'rb') as ifh:
    examples = pickle.load(ifh)

In [ ]:

all_letters = string.ascii_lowercase

plt.rcParams.update({'font.size': 5})
plt.rcParams.update({
"text.usetex": True,
"font.family": "sans-serif",
"font.sans-serif": ["Helvetica"]})


threshold = 0.1
n_range = range(0,250)
window_width_range = [2*n+1 for n in n_range]
linewidth=1
year_of_data = 2019

examples_chunks = [examples[0:2], examples[2:4], examples[4:]]


for chunk_number, example_chunk in enumerate(examples_chunks):

    
    fig, axs = plt.subplots(len(example_chunk),3, figsize=(6.5,2*len(example_chunk)), constrained_layout=True)
    # fig.tight_layout()

    for ex_no, example in enumerate(example_chunk):

        
        
        hr_start = example['hr_start']
        hr_end = example['hr_end']
        day=example['day']
        month=example['month']
        
        mode = 'constant'

        corr_results= {}

        X_o = example['obs']
        X_f = example['forecast']

        # Evaluate mean of truth array and make sure fcst_array has the same mean.

        f0 = (X_o > 0.1).mean()
        fss_results = [fss(X_o=X_o[:,:], X_f=X_f[:,:], thr=threshold, 
                        scale=2*n+1) for n in n_range] 
        
        fssim_bias_beta=3.0
        fssim_rhomin=0.1

        fssim_results = [calculate_fractions_SSIM(X_o=X_o[:,:], X_f=X_f[:,:], thr=threshold, 
                        scale=2*n+1, max_bias=fssim_bias_beta, rho_min=fssim_rhomin, mode=mode) for n in n_range]
        fssim_scores = [item[0] for item in fssim_results]
        fssim_thresholds = [item[1] for item in fssim_results]


        S_f = [get_convoluted_fractions(array=X_f[:,:], threshold=threshold, scale=2*n+1, mode=mode) for n in n_range]
        S_o = [get_convoluted_fractions(array=X_o[:,:], threshold=threshold, scale=2*n+1, mode=mode) for n in n_range]

        fcst_mu = [item.mean() for item in S_f]
        fcst_sigma = [item.std() for item in S_f]

        obs_mu = [item.mean() for item in S_o]
        obs_sigma = [item.std() for item in S_o]


        rho = [pearsonr(S_f[ii].flatten(), S_o[ii].flatten()).statistic for ii in range(len(n_range))]
        mu_bias = [2*obs_mu[ii]*fcst_mu[ii] / (obs_mu[ii]**2 + fcst_mu[ii]**2) for ii in range(len(n_range))]
        sigma_bias = [2*obs_sigma[ii]*fcst_sigma[ii] / (obs_sigma[ii]**2 + fcst_sigma[ii]**2) for ii in range(len(n_range))]
    
        fig2, ax = plt.subplots(1, 1, figsize=(3.5,3))
        
        ax.plot(window_width_range, rho, label=r'$\rho_n$', linewidth=linewidth, color='b', linestyle='dotted')
        ax.plot(window_width_range, fss_results, label=r'$\textrm{FSS}$', linewidth=linewidth, color='k')

        fcst_fss = fss_results

        ax.legend(loc='lower right')
        ax.set_xlabel(r'\textrm{Neighbourhood width (pixels)}')
        ax.set_ylim(min(min(rho) - 0.01, 0),1)
        ax.hlines(y=0.5 + f0*0.5, xmin=min(window_width_range), xmax=max(window_width_range), linestyles='dashed', colors='k', linewidth=linewidth)
        ax.set_xticks([1,200,400])
      
        plt.savefig(f'plots/fss_Ln_{month}_{day}_{year_of_data}_hr{hr_start}-{hr_end}_thr{threshold}_mode-{mode}.pdf', format='pdf', bbox_inches = 'tight')

        axs[ex_no, 0].plot(window_width_range, fssim_scores, label=r'$\textrm{F-SSIM}$', linewidth=linewidth, color='k', linestyle='-')
        axs[ex_no, 0].plot(window_width_range, fss_results, label=r'$\textrm{FSS}$', linewidth=linewidth, color='r')
        axs[ex_no, 0].plot(window_width_range, fssim_thresholds, linewidth=linewidth, linestyle='--')

        axs[ex_no, 0].legend(loc='lower right', ncol=2)
        axs[ex_no, 0].set_xlabel(r'\textrm{Neighbourhood width (pixels)}')
        axs[ex_no, 0].set_ylim(0,1)
        axs[ex_no, 0].set_xlim(0,max(window_width_range))
        axs[ex_no, 0].set_xticks([1,100,200,300,400,500])
        axs[ex_no, 0].set_title(f'({all_letters[ex_no * 3]})')

        # F-SSIM plots
        axs[ex_no, 1].plot(window_width_range,rho, label=r'$\rho_n$', linewidth=linewidth, color='b', linestyle='dotted')
        axs[ex_no, 1].plot(window_width_range,mu_bias, label=r'$\frac{2\mu_o \mu_f}{ (\mu^2_o + \mu^2_f)}$', linewidth=linewidth, color='k', linestyle='--')
        axs[ex_no, 1].plot(window_width_range,sigma_bias, label=r'$\frac{2\sigma_o \sigma_f }{ (\sigma^2_o + \sigma^2_f)}$', linewidth=linewidth, color='r', linestyle='-.')

        if (chunk_number ==0 and ex_no ==1) or (chunk_number == 2):
            ncol=1
        else:
            ncol = 2
        axs[ex_no, 1].legend(loc='lower right', ncol=ncol)
        axs[ex_no, 1].set_ylim(min(min(rho),0),1)
        axs[ex_no, 1].set_xlim(0,max(window_width_range))
        axs[ex_no, 1].set_xticks([1,100,200,300,400,500])
        axs[ex_no, 1].set_xlabel(r'\textrm{Neighbourhood width (pixels)}')
        axs[ex_no, 1].set_title(f'({all_letters[ex_no * 3 + 1]})')

        corr, p = calculate_correlation(X_o, X_f, threshold=threshold,
                                    window_range=[1,2,3,4,5, 7, 10, 15, 20, 25, 30, 40, 50, 60, 80, 100, 150, 200, 230])
        obs_mean = 0.5*(np.array(list(corr['t_tn_row'].values())) + np.array(list(corr['t_tn_col'].values())))
        fcst_mean = 0.5*(np.array(list(corr['f_fn_row'].values())) + np.array(list(corr['f_fn_col'].values())))

        axs[ex_no, 2].plot(corr['t_tn_row'].keys(), obs_mean, linewidth=linewidth, color='k', linestyle='-', label='Observations (IMERG)')
        axs[ex_no, 2].plot(corr['t_tn_row'].keys(), fcst_mean, linewidth=linewidth, color='k', linestyle='--', label='Forecast')
        axs[ex_no, 2].set_xlabel(r'\textrm{Separation (pixels)}')
        axs[ex_no, 2].set_ylabel(r'\textrm{Averaged correlation}')
        axs[ex_no, 2].legend(loc='upper right')
        axs[ex_no, 2].set_xticks([1,100,200])
        axs[ex_no, 2].set_title(f'({all_letters[ex_no * 3 + 2]})')

        plt.figure(fig)
        plt.savefig(f'plots/fssim_fig_{chunk_number}.pdf', format='pdf', bbox_inches = 'tight')

In [ ]:

threshold = 0.1
n_range = range(1,250)

def fss_lower_bound(mu_obs, mu_f, sigma_obs, sigma_f):
    return (2*mu_obs*mu_f)/(mu_obs**2 + mu_f**2 + sigma_obs**2 + sigma_f**2)

def fss_correlation(fss, mu_obs, mu_f, sigma_obs, sigma_f):
    lb = fss_lower_bound(mu_obs, mu_f, sigma_obs, sigma_f)
    return (fss - lb) * (mu_obs**2 + mu_f**2 + sigma_obs**2 + sigma_f**2) / (2*sigma_f*sigma_obs)

def theoretical_fss(mu_obs, mu_f, sigma_obs, sigma_f, rho):
    
    lb = fss_lower_bound(mu_obs, mu_f, sigma_obs, sigma_f)
    
    return lb  + 2 *(rho * sigma_f * sigma_obs) / (mu_obs**2 + mu_f**2 + sigma_obs**2 + sigma_f**2)


n_random_samples = 20
n_range = np.arange(250)
window_width_range = [2*n+1 for n in n_range]
threshold=0.1

mode = 'constant'

corr_results= {}

with open('fss_bias_data_2019_3_1.pkl', 'rb') as ifh:
    X_o = pickle.load(ifh)

S_o = [get_convoluted_fractions(array=X_o[:,:], threshold=threshold, scale=2*n+1, mode='constant') for n in n_range]

obs_mu = [item.mean() for item in S_o]
obs_sigma = [item.std() for item in S_o]

bias_vals_mu = np.arange(0,3,1.0)
bias_vals_sigma = np.arange(-0.6, 0, 0.2)
theoretical_fss_vals_mu = []

for n, bv in enumerate(bias_vals_mu):
    
    rho_crit=0.1
    
    rand_mu = [(1+bv)*item for item in obs_mu]
    rand_sigma = [item for item in obs_sigma]

    theoretical_fss_vals_mu.append([theoretical_fss(obs_mu[n], rand_mu[n], obs_sigma[n], rand_sigma[n], rho_crit) for n in range(len(n_range))])
    
theoretical_fss_vals_sigma = []
for n, bv in enumerate(bias_vals_sigma):
    
    rho_crit=0.3
    
    rand_mu = [item for item in obs_mu]
    rand_sigma = [(1+bv)*item for item in obs_sigma]

    theoretical_fss_vals_sigma.append([theoretical_fss(obs_mu[n], rand_mu[n], obs_sigma[n], rand_sigma[n], rho_crit) for n in range(len(n_range))])

In [ ]:
plt.rcParams['text.usetex'] = True
plt.rcParams.update({'font.size': 10})

fig, axs = plt.subplots(1,2,figsize=(6,3))

linestyles=['-', '--',':']


for m, bv in enumerate(bias_vals_mu):
    
    axs[0].plot(window_width_range, theoretical_fss_vals_mu[m], label=r'$\beta_\mu$' + f'= {bv:0.1f}', color='k', linestyle=linestyles[m])
    
axs[0].set_xlabel('Neighbourhood width (pixels)')

# ax.plot(window_range, theoretical_random_fss_vals, label=f'rand_th')
axs[0].set_ylim([0,1.01])
axs[0].set_xticks([1,200,400])
axs[0].legend()

for m, bv in enumerate(bias_vals_sigma):
    
    axs[1].plot(window_width_range, theoretical_fss_vals_sigma[m], label=r'$\beta_\sigma$' + f'= {bv:0.1f}', color='k', linestyle=linestyles[m])
    
axs[1].set_xlabel('Neighbourhood width (pixels)')

# ax.plot(window_range, theoretical_random_fss_vals, label=f'rand_th')
axs[1].set_ylim([0,1.01])
axs[1].legend(loc='lower right')
axs[1].set_xticks([1,200,400])

plt.savefig('plots/bias.pdf', format='pdf', bbox_inches='tight')


In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from matplotlib import colors
import cartopy.feature as cfeature


lake_feature = cfeature.NaturalEarthFeature(
    'physical', 'lakes',
    cfeature.auto_scaler, edgecolor='black', facecolor='never')

def plot_contourf( data, title, lon_range, lat_range, ax=None, value_range=None,
                  cmap='Reds', extend: str='both'):
    
    
    if value_range is not None:
        im = ax.contourf(lon_range, lat_range, data, transform=ccrs.PlateCarree(),
                            cmap=cmap, 
                            levels=value_range, norm=colors.Normalize(min(value_range), max(value_range)),
                            extend=extend)
    else:

        im = ax.contourf(lon_range, lat_range, data, transform=ccrs.PlateCarree(),
                    cmap=cmap, 
                    extend=extend)

    ax.coastlines(resolution='10m', color='black', linewidth=0.4)
    
    ax.add_feature(lake_feature, alpha=0.4)
    ax.set_title(title)
    ax.set_ylabel('asdf')
    
    return im

In [ ]:
plt.rcParams.update({'font.size': 10})

from src.scoring import get_convoluted_fractions
from src.utils import plot_contourf
import cartopy.crs as ccrs
from matplotlib import colorbar, gridspec

 


n_vals = [0, 10, 100]

for example in examples[1:2]:

    fig = plt.figure(constrained_layout=True, figsize=(6.5, 4))
    gs = gridspec.GridSpec(3, len(n_vals), figure=fig, 
                        height_ratios=[1, 1] + [0.05],
                        wspace=0.005)  

    month = example['month']
    day = example['day']
    hr_start = example['hr_start']
    hr_end = example['hr_end']
    year_of_data = 2019
    threshold = 0.1
    mode = 'constant'
    print(month, day, year_of_data)


    for m, n_val in enumerate(n_vals):

        ax_obs = fig.add_subplot(gs[0, m], projection = ccrs.PlateCarree())
        ax_fcst = fig.add_subplot(gs[1, m], projection = ccrs.PlateCarree())
        
        width = 2*n_val + 1

        smoothed_obs = get_convoluted_fractions(example['obs'], threshold=threshold, scale=width, mode=mode)
        smoothed_fcst = get_convoluted_fractions(example['forecast'], threshold=threshold, scale=width, mode=mode)
        
        max_val = np.round(max(smoothed_obs[0,:,:].max(), smoothed_fcst[0,:,:].max()), 2)

        if max_val > 0.9:
            step_size = 0.2
        elif max_val < 0.2:
            step_size = 0.05
        else:
            step_size = 0.1

        val_range = [item for item in  np.arange(0,1+step_size,step_size/4) if item < max_val]
        if val_range[-1] != 1:
            val_range = val_range + [np.arange(0,1+step_size,step_size/4)[len(val_range)]]

        ticks = [item for item in  np.arange(0,val_range[-1]+0.1,step_size) ]

        smoothed_obs[0,:,:][smoothed_obs[0,:,:] ==0] = -0.1
        smoothed_fcst[0,:,:][smoothed_fcst[0,:,:] ==0] = -0.1
        im0 = plot_contourf(ax=ax_obs, data=smoothed_obs[0,:,:], title=f'Width={width}', lon_range=longitude_range, 
                            lat_range=latitude_range, cmap='Blues', value_range=val_range, extend='neither')
        im1 = plot_contourf(ax=ax_fcst, data=smoothed_fcst[0,:,:], title=f'Width={width}', lon_range=longitude_range, lat_range=latitude_range, 
                            cmap='Blues', value_range=val_range, extend='neither')

        cbar_ax = fig.add_subplot(gs[-1, m])
        cb0 = fig.colorbar(im0, cax=cbar_ax, orientation='horizontal', shrink = 0.2, aspect=8, ticks=ticks)
    
plt.savefig(f'plots/blurring_{month}_{day}_{year_of_data}_hr{hr_start}-{hr_end}_thr{threshold}_mode{mode}.pdf', format='pdf', bbox_inches = 'tight')
